In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git
!cd darknet
!make
!cd ..

In [ ]:
!wget https://pjreddie.com/media/files/yolov5.weights

In [ ]:
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# Load YOLOv3 model and weights
def LoadYoloModel():
    net = cv2.dnn.readNet("darknet/yolov5.weights", "darknet/cfg/yolov5.cfg")
    with open("darknet/data/coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    return net, classes

In [ ]:
# Preprocess image for YOLO
def PreProcessImage(imagePath):
    image = cv2.imread(imagePath)
    height, width, _ = image.shape
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), (0, 0, 0), True, crop=False)
    return image, height, width, blob

In [ ]:
# Perform object detection
def DetectObjects(net, blob, image):
    layerNames = net.getLayerNames()
    outputLayers = [layerNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    net.setInput(blob)
    layerOutputs = net.forward(outputLayers)
    return layerOutputs

In [ ]:
# Get detection results
def GetDetectionResults(height, width, layerOutputs, classes, confThreshold=0.5, nmsThreshold=0.4):
    class_ids = []
    confidences = []
    boxes = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > confThreshold:
                center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)

    return boxes, confidences, class_ids, indices


In [ ]:
# Draw detected objects on the image
def DrawDetectedObjects(image, classes, boxes, confidences, class_ids, indices):
    colors = np.random.uniform(0, 255, size=(len(classes), 3))

    for i in indices:
        i = i[0]
        box = boxes[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        label = str(classes[class_ids[i]])
        confidence = confidences[i]
        color = colors[class_ids[i]]
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image, f"{label} {confidence:.2f}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        return image

In [ ]:
def main(image_path, output_path):
    net, classes = LoadYoloModel()
    image, height, width, blob = PreProcessImage(image_path)
    layer_outputs = DetectObjects(net, blob, image)
    boxes, confidences, class_ids, indices = GetDetectionResults(height, width, layer_outputs, classes)
    detected_image = DrawDetectedObjects(image, classes, boxes, confidences, class_ids, indices)
    cv2.imwrite(output_path, detected_image)


In [ ]:
def LoadDataSet(dataSetFolderPath: str) -> tuple:
    annotationsFilePath = dataSetFolderPath+"/annotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=",")
    annotations = []
    for index, row in annotationsDataFrame[1:].iterrows():
        annotations.append(
            dataSetFolderPath+"/"+row[0])

    del annotationsDataFrame
    X_train, X_val, y_train, y_val = train_test_split(
        annotations, annotations, test_size=0.2, random_state=42)
    return  X_train, X_val, y_train, y_val

In [ ]:
X_train, X_val, y_train, y_val = LoadDataSet("./DataSet1")
for path in X_val:
    main(path, path.split("/")[-1])